In [ ]:
#Libraries
using Gen
using PyPlot
using Distributions
using LinearAlgebra
using Flux
using Random
using Distances
include("hmc_mod.jl")
include("helper_functions.jl")
include("rj_proposals.jl")
include("NUTS.jl");

In [ ]:
#------------------------------------
#Hyperparameters and Helper Functions
#------------------------------------

#Select Network Goal
network = "classifier"
#network = "interpolator"

#Data hyperparameters
n = 20 #Number of samples per mode (classifier)
m = 4 #Number of modes (classifier)
d = 2 #Input dimension
N = n*m #Total samples
σₐ = 0.02 #Mode variance (classifier)

#Network hyperparameters
α = 6 #Gamma Scale for Hyperparameters

#Node hyperparameters
k_range = 12 #Maximum number of neurons per layer
k_list = [Int(i) for i in 1:k_range]
k_real = 2

#Layer hyperparameters
l_range = 1 #Maximum number of layers in the network
l_list = [Int(i) for i in 1:l_range]
l_real = 1

#NUTS
Δmax = 1000;

In [ ]:
#Data
x_raw, classes = real_data_classifier(Int(N/4), 4, σₐ);
classes = [(i+1) % 2 + 1 for i in classes]
y_real = classes

plot_data_classifier(x_raw,classes)
x = transpose(x_raw)
size(x);

In [ ]:
#Bayesian Neural Net
function G(x, trace)
    activation = relu
    layers = trace[:l]
    ks = [trace[(:k,i)] for i=1:layers]
    for i=1:layers
        in_dim, out_dim = layer_unpacker(i, layers, ks)
        W = reshape(trace[(:W,i)], out_dim, in_dim)
        b = reshape(trace[(:b,i)], trace[(:k,i)])
        nn = Dense(W, b, activation)
        x = nn(x)
    end
    
    Wₒ = reshape(trace[(:W,layers+1)], 1, ks[layers])
    bₒ = reshape(trace[(:b,layers+1)], 1)
    
    nn_out = Dense(Wₒ, bₒ)
    return nn_out(x)
end;

@gen function classifier(x::Array{Float64})
    
    #Create a blank choicemap
    obs = choicemap()::ChoiceMap
    
    #Draw number of layers
    l ~ categorical([1/length(l_list) for i=1:length(l_list)])
    l_real = l
    obs[:l] = l
    
    #Create individual weight and bias vectors
    #Loop through hidden layers
    k = [Int(0) for i=1:l+1]
    for i=1:l
        k[i] = @trace(categorical([1/length(k_list) for i=1:length(k_list)]), (:k,i))
        obs[(:k,i)] = k[i]
    end
    k[l+1] = @trace(categorical([1.0]), (:k,l+1))
    obs[(:k,l+1)] = k[l+1]
    
    α = 0.001 #Gamma Scale for Hyperparameters
    
    ω₁ = 100
    ω₂ = (sum([obs[(:k,i)] for i=1:l]))*100 #Neal (1996): Scaling relationship to # of hidden units
    τ₁ ~ gamma(ω₁,α) #Hidden Weights
    τ₂ ~ gamma(ω₁,α) #Hidden Biases
    τ₃ ~ gamma(ω₂,α) #Output Weights
    #τᵧ ~ gamma(ωᵧ,α) #Noise Parameter for y
    #τ₄ ~ gamma() #Output Biases - Neal uses fixed sigmas here
    
    #Standard Deviations
    σ₁ = 1/τ₁
    σ₂ = 1/τ₂
    σ₃ = 1/τ₃
    #σᵧ = sqrt(1/τᵧ)
    
    #Sample weight and parameter vectors
    W = [zeros(k[i]) for i=1:l+1]
    b = [zeros(k[i]) for i=1:l+1]
    for i=1:l+1
        if i == 1
            h = Int(d * k[i])
        else
            h = Int(k[i-1] * k[i])
        end

        if i<=l
            #Hidden Weights
            μ = zeros(h)
            Σ = Diagonal([σ₁ for i=1:length(μ)])
            W[i] = @trace(mvnormal(μ,Σ), (:W,i))
            obs[(:W,i)] = W[i]
            
            #Hidden Biases
            μ2 = ones(k[i])
            Σ2 = Diagonal([σ₂ for i=1:length(μ2)])
            b[i] = @trace(mvnormal(μ2,Σ2), (:b,i))
            obs[(:b,i)] = b[i]
        else
            #Output Weights
            μₒ = zeros(k[l])
            Σₒ = Diagonal([σ₃ for i=1:length(μₒ)])
            W[i] = @trace(mvnormal(μₒ,Σₒ), (:W,i))
            obs[(:W,i)] = W[i]

            #Output Bias
            μ2ₒ = ones(1)
            Σ2ₒ = Diagonal([1.0 for i=1:length(μ2ₒ)])
            b[i] = @trace(mvnormal(μ2ₒ,Σ2ₒ), (:b,i))
            obs[(:b,i)] = b[i]
        end
    end
    
    #Return Network Scores for X
    scores = G(x,obs)
    scores = Flux.σ.(scores)
    
    #Logistic Regression Likelihood
    y = zeros(length(scores))
    for j=1:N
        y[j] = @trace(categorical([1-scores[j],scores[j]]), (:y,j))
    end

    return scores
    
end;

#(best_trace,) = generate(classifier, (x,), obs)
#println(best_trace[:τ₁])
#println(best_trace[:τ₂])
#println(best_trace[:τ₃])

test_scores = classifier(x)
test_labels = data_labeller(test_scores)
test_acc = sum([classes[i] == test_labels[i] for i=1:length(classes)])

In [ ]:
#-----------------------------------------
#Register Observed Data - Test Likelihood
#-----------------------------------------
obs_master = choicemap()::ChoiceMap
for i=1:length(classes)
    obs_master[(:y,i)] = classes[i]
end
obs = obs_master;

scores = []
accs = []
ks = []
best_ks = []
best_traces = []
(best_trace,) = generate(classifier, (x,), obs)
best_acc = 0
best_score = get_score(best_trace)
best_pred_y = (G(x, best_trace))
best_pred_labels = data_labeller(best_pred_y)
best_k = best_trace[(:k,1)]
while best_acc <= 40
    (best_trace,) = generate(classifier, (x,), obs)
    best_score = get_score(best_trace)
    best_k = best_trace[(:k,1)]
    best_pred_y = (G(x, best_trace))
    best_pred_labels = data_labeller(best_pred_y)
    #best_pred_y = [best_trace[(:y,i)] for i=1:length(classes)]
    best_acc = sum([classes[i] == best_pred_labels[i] for i=1:length(classes)])
end;
#println(best_trace[:τ₁])
#println(best_trace[:τ₂])
#println(best_trace[:τ₃])
#println(best_trace[:τᵧ])

println(best_acc)
println(get_score(best_trace))

In [ ]:
#----------------
#Test Likelihood
#----------------
scores = []
accs = []
ks = []
best_ks = []
best_traces = []
function likelihood(best_trace, best_acc, best_score, best_k)
    obs = obs_master;
    #obs[(:k,1)] = 2
    (trace,) = generate(classifier, (x,), obs)
    
    pred_y = (G(x, trace))
    pred_labels = data_labeller(pred_y)
    acc = sum([classes[i] == pred_labels[i] for i=1:length(classes)])
    
    #pred_y = [trace[(:y,i)] for i=1:length(classes)]
    #acc = sum([classes[i] == pred_y[i] for i=1:length(classes)])
    score = get_score(trace)
    #(score, retval) = assess(lh, (x,trace), obs_master)
    
    #println(score)
    if acc > best_acc
        best_acc = acc
        best_score = score
        best_trace = trace
        best_pred_y = pred_y
        best_k = best_trace[(:k,1)]
    end
    #push!(best_ks,best_k)
    push!(scores,score)
    push!(accs,acc)
    #push!(ls, l)
    #println(best_score)
    return(best_trace, best_acc, best_score, best_k)
end;

for i=1:10000
    best_trace, best_acc, best_score, best_k = likelihood(best_trace, best_acc, best_score, best_k)
    push!(best_ks, best_k)
end

PyPlot.scatter(accs, scores)
plt.title("Comparing Classifier Accuracy to Log Likelihood")
plt.xlabel("Classifier Accuracy")
plt.ylabel("Log Likelihood")
plt.ylim(-250,1)
plt.legend()
#print(best_ks)

In [ ]:
obs_master = choicemap()::ChoiceMap
for i=1:length(classes)
    obs_master[(:y,i)] = classes[i]
end
obs = obs_master;
(trace,) = generate(classifier, (x,), obs);
args = get_args(trace)
retval_grad = accepts_output_grad(get_gen_fn(trace)) ? zero(get_retval(trace)) : nothing
argdiffs = map((_) -> NoChange(), args)
selection = select_selection_NUTS(trace)
gen_fn = trace.gen_fn
state = GFBackpropTraceState(trace, selection, gen_fn.params, tape)

In [ ]:
#--------------------
#RJMCMC - using NUTS
#--------------------
traces = []
scores = []
scores2 = []
acc = []
acc_a = []
acc_w = []
acc_l = []
k_results = []
epss = []
aL = 20
wL = 1

function within_move(trace, iters, obs, prev_trace)
    for i=1:iters
        eps = uniform(0.01,0.04)
        selection = select_selection(trace)
        (new_trace, hmc_score) = NUTS(trace, selection, eps, false, obs, 1, prev_trace)
        #println(hmc_score)
        if rand(Uniform(0,1)) < exp(hmc_score)
            trace = new_trace
            accepted = true
            #println("Accepted")
        else
            trace = prev_trace
            accepted = false
            #println("Not Accepted")
        end
        push!(traces, trace)
        push!(acc, accepted)
        push!(acc_w, accepted)
        #accepted && println("Within accepted")
    end
    return trace
end

function node_move(trace, iters, obs, prev_trace)
    
    #Determine birth or death
    current_l = trace[:l]
    layer = rand((1:current_l))
    current_k = trace[(:k, layer)]
    
    if current_k == last(k_list)
        move_type = 0
    elseif current_k == k_list[1]
        move_type = 1
    else
        move_type = bernoulli(0.5)
    end
    move = "Empty"
    
    obs_master = choicemap()::ChoiceMap
    for i=1:length(classes)
        obs_master[(:y,i)] = classes[i]
    end
    
    eps = rand(Uniform(0.1,0.4))

    #HMC Move 1
    selection = select_selection(trace)
    hmc1_trace = trace
    (hmc1_trace, hmc1_score) = NUTS(hmc1_trace, selection, eps, false, obs, 2, prev_trace)

    #RJ Move
    if move_type == 1
        move = "Birth"
        rj_trace = node_birth(hmc1_trace, layer)
    else
        move = "Death"
        rj_trace = node_death(hmc1_trace, layer)
    end

    #HMC Move 2
    hmc2_trace = rj_trace
    (hmc2_trace, hmc2_score) = NUTS(hmc2_trace, selection, eps, false, obs, 2, hmc2_trace)

    #(score1,) = assess(lh, (x, trace), obs_master)
    #(score2,) = assess(lh, (x, hmc2_trace), obs_master)
    score1 = get_score(trace)
    score2 = get_score(hmc2_trace)
    logscore = (score2 - score1)
    score = exp(logscore) * exp(hmc1_score) * exp(-hmc2_score)
    #println("$move: $score")
    
    if rand(Uniform(0,1)) < score
        accepted = true
        trace = hmc2_trace
        new_k = [trace[(:k,i)] for i=1:trace[:l]]
        println("New ks accepted! Current ks: $(new_k)")
    else
        println("Sticking with the old k!")
        accepted = false
        trace = prev_trace
    end

    #println("$move Old Trace: $score1; Pre-HMC: $score_test; Post-HMC: $score2")
        
    push!(traces, trace)
    push!(acc, accepted)
    push!(acc_a, accepted)
    push!(scores, score)
    push!(scores2, logscore)
    return trace
end


function rjmcmc(starting_trace, iters)
    trace = starting_trace
    l = trace[:l]
    ks = [trace[(:k,i)] for i=1:trace[:l]]
    println("Beginning RJMCMC")
    println("Starting ks: $ks")
    println("--------------------------------")

    for i=1:iters
        l = trace[:l]
        ks = [trace[(:k,i)] for i=1:trace[:l]]
        #obs_master = choicemap()::ChoiceMap
        #for i=1:length(classes)
            #obs_master[(:y,i)] = classes[i]
        #end
        obs = obs_master;
        if i%50 == 0
            #println("Epoch $i Acceptance Prob: $(sum(acc)/length(acc))")
            #println("Epoch $i layer count: $l, ks: $ks")
            println("Epoch $i Within Acceptance Prob: $(sum(acc_w)/length(acc_w))")
            println("Epoch $i Across Acceptance Prob: $(sum(acc_a)/length(acc_a))")
            #println("Epoch $i Layer Acceptance Prob: $(sum(acc_l)/length(acc_l))")
            #println([trace[(:k,i)] for i=1:trace[:l]])
        end
        
        #Gibbs sampling for hyperparameters
        prev_trace = trace
        trace, obs = select_hyperparameters(prev_trace, obs)
        
        #Indicator variable for move type
        u = rand(Uniform(0,1))
        if u > 0.9
            (trace) = node_move(trace, 1, obs, prev_trace)
        else
            (trace) = within_move(trace, 1, obs, prev_trace)
        end
        push!(scores, get_score(trace))
        push!(k_results, trace[(:k,1)])
    end
    println("Finished")
end
 
runs = 100
#starting_trace = best_trace

#x_samp, y_samp = sample_data(x, y_real)
#x_samp = transpose(x_samp)
obs_master = choicemap()::ChoiceMap
for i=1:length(classes)
    obs_master[(:y,i)] = classes[i]
end
obs = obs_master
obs[:l] = 1
(starting_trace,) = generate(classifier, (x,), obs)
#starting_trace = best_trace
obs[:l] = starting_trace[:l]

rjmcmc(starting_trace,runs);

In [ ]:
include("NUTS.jl");

In [ ]:
plot(k_results)

In [ ]:
plot(scores2, c="Red")
#plot(scores, c="Blue")

In [ ]:
scores2 = [get_score(trace) for trace in traces]
plot(scores2)

In [ ]:
accs = []
for i=1:length(traces)
    trace = traces[i]
    pred_y = G(x,trace)
    pred_labels = data_labeller(pred_y)
    
    acc = acc = sum([classes[i] == pred_labels[i] for i=1:length(classes)])
    push!(accs,acc)
end

plot(accs)
println(sum(accs)/length(accs))
plt.title("RJMCMC Accuracy: XOR Classifier")
plt.xlabel("Iteration")
plt.ylabel("# Classified Correctly (out of 200)");

In [ ]:
#Bar Plot of k estimates
#ks = [traces[i][:k] for i in 1:length(traces)]
counts = [sum([k_results[i] == j for i in 1:length(k_results)]) for j in k_list]
#actual_ks = [i for i=1:maximum(ks)]
barlist=bar(k_list,counts)
for i in 1:k_range
    barlist[i].set_color([0.8,0.0,0.0])
end
barlist[2].set_color([0.0,0.8,0.1])

plt.title("Counts of Values for k from RJMCMC Trace: XOR Classifier")
plt.xlabel("k value")
plt.ylabel("Occurences");

In [ ]:
function plot_grid(data,scores,alpha=1.0)
    PyPlot.scatter(data[:,1],data[:,2],c=scores,alpha=alpha,cmap="PRGn")
    #PyPlot.colorbar()
end

function tracegrid(traces, samples=1000, low=-2.0, high=2.0)
    d=2
    n=100
    r = range(low, high, length = n)
    
    iter = Iterators.product((r for _ in 1:d)...)
    grid= vec([collect(i) for i in iter])
    grid_raw = reduce(hcat, getindex.(grid,i) for i in eachindex(grid[1]))
    grid2 = transpose(grid_raw)
    z_master = zeros(length(grid2[1,:]))
    
    for i=1:samples
        j = rand((100,length(traces)))
        trace = traces[j]
        z = Flux.σ.(G(grid2,trace))[1,:]
        z_master += (z ./ samples)
    end
    plot_grid(grid_raw, z_master)
end

tracegrid(traces)
plot_data_classifier(x_raw,classes)

In [ ]:
#-------------------------
#RJMCMC - Layers and Nodes
#OLD HMC BASED ALGORITHM
#-------------------------
traces = []
scores = []
scores2 = []
acc = []
acc_a = []
acc_w = []
acc_l = []
k_results = []
epss = []
aL = 20
wL = 1

function within_move(trace, iters, obs)
    for i=1:iters
        eps = uniform(0.01,0.4)
        selection = select_selection(trace)
        (new_trace, hmc_score) = hmc_mod(trace, selection, L=wL, eps=eps, check=false, observations=obs)
        if rand(Uniform(0,1)) < exp(hmc_score)
            trace = new_trace
            accepted = true
        else
            accepted = false
        end
        push!(traces, trace)
        push!(acc, accepted)
        push!(acc_w, accepted)
        #accepted && println("Within accepted")
    end
    return trace
end

function node_move(trace, iters, obs)
    
    #Determine birth or death
    current_l = trace[:l]
    layer = rand((1:current_l))
    current_k = trace[(:k, layer)]
    
    if current_k == last(k_list)
        move_type = 0
    elseif current_k == k_list[1]
        move_type = 1
    else
        move_type = bernoulli(0.5)
    end
    move = "Empty"
    
    obs_master = choicemap()::ChoiceMap
    for i=1:length(classes)
        obs_master[(:y,i)] = classes[i]
    end
    
    eps = rand(Uniform(0.01,0.4))
    
    #Can only iter once for now

    #HMC Move 1
    selection = select_selection(trace)
    hmc1_trace = trace
    (hmc1_trace, hmc1_score) = hmc_mod(hmc1_trace, selection, L=aL, eps=eps, check=true, observations=obs)

    #RJ Move
    if move_type == 1
        move = "Birth"
        rj_trace = node_birth(hmc1_trace, layer)
    else
        move = "Death"
        rj_trace = node_death(hmc1_trace, layer)
    end

    #HMC Move 2
    hmc2_trace = rj_trace
    (hmc2_trace, hmc2_score) = hmc_mod(hmc2_trace, selection, L=aL, eps=eps, check=true, observations=obs)

    #(score1,) = assess(lh, (x, trace), obs_master)
    #(score2,) = assess(lh, (x, hmc2_trace), obs_master)
    score1 = get_score(trace)
    score2 = get_score(hmc2_trace)
    logscore = (score2 - score1)
    score = exp(logscore) * exp(hmc1_score) * exp(-hmc2_score)
    #println("$move: $score")
    
    if rand(Uniform(0,1)) < score
        accepted = true
        trace = hmc2_trace
        new_k = [trace[(:k,i)] for i=1:trace[:l]]
        println("New ks accepted! Current ks: $(new_k)")
    else
        #println("Sticking with the old k!")
        accepted = false
    end

    #println("$move Old Trace: $score1; Pre-HMC: $score_test; Post-HMC: $score2")
        
    push!(traces, trace)
    push!(acc, accepted)
    push!(acc_a, accepted)
    push!(scores, score)
    push!(scores2, logscore)
    return trace
end


function rjmcmc(starting_trace, iters)
    trace = starting_trace
    l = trace[:l]
    ks = [trace[(:k,i)] for i=1:trace[:l]]
    println("Beginning RJMCMC")
    println("Starting ks: $ks")
    println("--------------------------------")
    #@showprogress 1 "Sampling..." for i = 1:iters
    for i=1:iters
        l = trace[:l]
        ks = [trace[(:k,i)] for i=1:trace[:l]]
        #x_samp, y_samp = sample_data(x, y_real)
        #x_samp = transpose(x_samp)
        obs_master = choicemap()::ChoiceMap
        for i=1:length(classes)
            obs_master[(:y,i)] = classes[i]
        end
        obs = obs_master;
        if i%100 == 0
            #println("Epoch $i Acceptance Prob: $(sum(acc)/length(acc))")
            #println("Epoch $i layer count: $l, ks: $ks")
            #println("Epoch $i Within Acceptance Prob: $(sum(acc_w)/length(acc_w))")
            println("Epoch $i Across Acceptance Prob: $(sum(acc_a)/length(acc_a))")
            #println("Epoch $i Layer Acceptance Prob: $(sum(acc_l)/length(acc_l))")
            println([trace[(:k,i)] for i=1:trace[:l]])
        end
        
        #Gibbs sampling for hyperparameters
        trace, obs = select_hyperparameters(trace, obs)
        
        #Indicator variable for move type
        u = rand(Uniform(0,1))
        if u > 0.0
            (trace) = node_move(trace, 1, obs)
        else
            (trace) = within_move(trace, 1, obs)
        end
        push!(scores, get_score(trace))
        push!(k_results, trace[(:k,1)])
    end
    println("Finished")
end
 
runs = 20000
#starting_trace = best_trace

#x_samp, y_samp = sample_data(x, y_real)
#x_samp = transpose(x_samp)
obs_master = choicemap()::ChoiceMap
for i=1:length(classes)
    obs_master[(:y,i)] = classes[i]
end
obs = obs_master
obs[:l] = 1
(starting_trace,) = generate(classifier, (x,), obs)
#starting_trace = best_trace
obs[:l] = starting_trace[:l]

rjmcmc(starting_trace,runs);